In [1]:
from sklearn.datasets import samples_generator
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier

# 학습 파이프라인

## 개념
- 하나의 분석 과정은 전처리/분류/특징 선택/모델 학습/예측 등의 여러 프로세스(모듈)로 이루어져있다.
- 모든 프로세스를 따로 접근하지 않고, 하나의 모듈로 통합 관리하는 것이 바로 학습 파이프라인

In [2]:
X, Y = samples_generator.make_classification(n_samples=150, n_features=25, n_classes=3, 
                                             n_informative=6, n_redundant=0, random_state=7)
#classification 위한 임의의 데이터셋을 만들어내는 메소드
#차례로, 만들 데이터 행/열, target 변수의 수, 
#열 중에 몇 개가 정보를 많이 함축하게 할지, 몇 개를 중복된 차원(선형 변환으로 끼리 큰 영양가 없는)를 선택하는 인자

In [3]:
#1) feature selector module
k_best_selector = SelectKBest(f_regression, k=9)

In [4]:
#2) classification module
classifier = ExtraTreesClassifier(n_estimators=60, max_depth=4) #number of trees, depth for pruning

In [5]:
#Making pipeline
processor_pipeline = Pipeline([("selector", k_best_selector), ("erf", classifier)])

- Pipeline은 각 모듈에 대한 정보(tuple)를 원소로 갖는 리스트를 인자로 받는다.
- 각 tuple은 해당 모듈을 지칭할 이름과 앞서 생성한 해당 모듈을 원소로 갖는다.

In [6]:
#parameter setting
processor_pipeline.set_params(selector__k=7, erf__n_estimators=30)
#'앞서 지정한 이름__인자 이름 = 값'의 형태로 전달

Pipeline(steps=[('selector', SelectKBest(k=7, score_func=<function f_regression at 0x1076471e0>)), ('erf', ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=4, max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=30, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False))])

In [7]:
#learning
processor_pipeline.fit(X, Y)

Pipeline(steps=[('selector', SelectKBest(k=7, score_func=<function f_regression at 0x1076471e0>)), ('erf', ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=4, max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=30, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False))])

In [8]:
#prediction and check
output = processor_pipeline.predict(X)
print("\nPredicted output:\n", output)


Predicted output:
 [0 2 2 0 2 0 2 1 0 1 1 2 0 0 2 2 1 0 0 0 0 2 1 1 2 2 0 0 2 2 1 2 1 0 2 2 1
 1 2 2 2 0 1 2 2 1 2 2 1 0 1 2 2 2 2 0 2 2 0 2 2 0 1 0 2 2 0 1 1 2 0 0 0 2
 0 0 1 2 2 0 0 1 2 2 2 0 0 0 2 2 2 1 2 0 2 0 2 2 0 0 1 1 1 1 2 2 2 2 0 1 1
 0 2 1 0 0 1 1 1 1 0 0 0 1 2 0 0 0 2 1 2 0 0 1 0 1 1 0 1 1 1 1 2 0 0 1 2 0
 2 2]


In [9]:
#pipeline score
print("\nScore:", processor_pipeline.score(X,Y))


Score: 0.88


- 파이프라인 스코어는 예측 결과를 이용해 산정한 값. (책에 파이프라인 효율성이라길래 프로세스 연결 구조를 스코어링하는 줄 알았....)

In [10]:
#extracting top features using k_best_selector
status = processor_pipeline.named_steps["selector"].get_support()

In [11]:
status

array([False, False, False, False,  True, False, False,  True,  True,
       False, False, False,  True, False,  True, False, False,  True,
       False, False, False, False,  True, False, False], dtype=bool)

In [12]:
#check extracted features
selected = [i for i,x in enumerate(status) if x]
print("\nIndices of selected features:", ', '.join([str(x) for x in selected]))


Indices of selected features: 4, 7, 8, 12, 14, 17, 22
